In [ ]:
# Importar librerías
import pandas as pd # Se usa para la lectura y escritura de datos, manipulación de marcos de datos, limpieza y preprocesamiento de datos.
import numpy as np # Se usa para manipulación de matrices, álgebra lineal, generación de números aleatorios.
import matplotlib.pyplot as plt # Se usa para creación de gráficos de líneas, histogramas, dispersión.
import seaborn as sns # Se usa para crear visualizaciones estadísticas más atractivas y detalladas.
from wordcloud import WordCloud # Se usa para la visualización de palabras clave en función de su frecuencia en un conjunto de texto.
import nltk # Se usa para entender y procesar el lenguaje escrito de una manera más fácil y efectiva
from typing import Dict
import warnings # Se usa para el control de advertencias para mejorar la legibilidad y depuración del código.
from sklearn.metrics.pairwise import cosine_similarity # Es un sistemas de recomendación, procesamiento de texto, y otros contextos donde se necesita medir la similitud entre elementos.
from sklearn.metrics.pairwise import linear_kernel # Es utilizada en contextos de aprendizaje automático y minería de datos.
from sklearn.feature_extraction.text import TfidfVectorizer # Preprocesamiento de texto para tareas de aprendizaje automático, como la clasificación de texto o la agrupación de documentos.
from collections import Counter

In [ ]:
# Lectura y visualización de archivos
# Df_General = pd.read_parquet('Df_General.parquet')
Df_muestra = pd.read_parquet('Df_muestra.parquet')
# Se crea una muestra de 20000 datos para poder realizar las consultas
# Df_muestra = Df_General.head(20000)
# Guardar la muestra en un nuevo archivo parquet
# Df_muestra.to_parquet('Df_Muestra.parquet', index=False)
# Guardar la muestra en un nuevo archivo CSV
# Df_muestra.to_csv('Df_Muestra.csv', index=False)
Df_muestra.head(1)
# Df_General['id'].unique()
# Df_General['year'].unique()
# Df_General.info()

CREACIÓN DE FUNCIONES

In [ ]:
# Función N°1. Proporciona el año en el que se registró la mayor cantidad de horas de juego para un género específico.
def PlayTimeGenre(genre: str):
    # Filtrar los datos por el género proporcionado
    df_genres = Df_muestra[Df_muestra[genre] == 1]
    # Agrupar por año y calcular la suma de las horas jugadas
    sum_year_playtime = df_genres.groupby('release_year')['playtime_forever'].sum()
    # Encontrar el año con la mayor suma de horas jugadas
    max_year = sum_year_playtime.idxmax()
    # Retornar el resultado en un diccionario
    result = {
        "Año de lanzamiento con más horas jugadas para género": genre,
        "Año": int(max_year),
        "Horas Jugadas": int(sum_year_playtime[max_year])
    }
    return result

PlayTimeGenre('Action')

In [ ]:
# Función N°2. Devuelve el usuario que tiene la mayor cantidad de horas de juego en un género específico
def UserForGenre(genero: str) -> dict:
    df_genres = Df_muestra[Df_muestra[genero] == 1]
    agg_df = df_genres.groupby('release_year').agg({'user_id': 'max', 'playtime_forever': 'sum'}).reset_index()
    playtime_list = agg_df.to_dict(orient='records')

    result = {
        "Usuario con más horas jugadas para Género " + genero: 
        df_genres.loc[df_genres['playtime_forever'].idxmax(), 'user_id'],
        "Horas jugadas": playtime_list
    }
    return result
UserForGenre('Action')

In [15]:
# Función N°3 Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
def UsersRecommend(year : int):

    if year >= 2010 and year <= 2015:
        filtrado = Df_muestra[Df_muestra['year'] == year]
        filtrado = filtrado[filtrado['sentiment'].isin([1, 2])]
        conteo = filtrado.groupby('app_name')['recommend'].sum().reset_index()
        top_3 = conteo.nlargest(3, 'recommend')
        return {
            f"Top 3 de los juegos MÁS recomendados por el año {year}":
            [
                {'1': f"Name = {str(top_3['app_name'].iloc[0])}"},
                {'2': f"Name = {str(top_3['app_name'].iloc[1])}"},
                {'3': f"Name = {str(top_3['app_name'].iloc[2])}"}
            ]
        }
    else:
        return {f'Año no encontrado'}
    
UsersRecommend(2012)

{'Top 3 de los juegos MÁS recomendados por el año 2012': [{'1': "Name = Garry's Mod"},
  {'2': 'Name = Day of Defeat: Source'},
  {'3': 'Name = Half-Life'}]}

In [10]:

# Función N°4 Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado.
def UsersWorstDeveloper(year: int):
    # Filtra el DataFrame por el año dado
    filtrado = Df_muestra[Df_muestra['year'] == year]
    # Agrupa por desarrolladora y calcula la media de las recomendaciones
    df_agrupado = filtrado.groupby('developer')['recommend'].mean().reset_index()
    # Ordena en orden ascendente (menor recomendación primero) y toma las primeras 3 desarrolladoras
    top3_desarrolladoras = df_agrupado.sort_values(by='developer').head(3)

    return {"top3_desarrolladoras_menos_recomendadas": top3_desarrolladoras.to_dict(orient='records')}
UsersWorstDeveloper(2010)

{'top3_desarrolladoras_menos_recomendadas': [{'developer': 'Facepunch Studios',
   'recommend': 1.0},
  {'developer': 'Valve', 'recommend': 1.0}]}

In [16]:
# Función N°5 Analisis de sentimiento
def sentiment_analysis(año_de_lanzamiento:int) -> Dict[int, Dict[str, int]]:
        df = Df_muestra[Df_muestra['release_year'] == año_de_lanzamiento]
        sentiment_counts = df['sentiment'].value_counts()
        result = {año_de_lanzamiento: {
            'Negative': sentiment_counts.get(0, 0),
            'Neutral' : sentiment_counts.get(1, 0),
            'Positive': sentiment_counts.get(2, 0)
        }}

        return result
sentiment_analysis(2007)

{2007: {'Negative': 54, 'Neutral': 42, 'Positive': 146}}

MODELO DE MACHINE LEARNING - Recomendación user-item

In [20]:
# Se crea el modelo de machine learning con Scikit-Learn
tfidf = TfidfVectorizer(stop_words='english')
Df_muestra=Df_muestra.fillna("")

tdfid_matrix = tfidf.fit_transform(Df_muestra['review'])
cosine_similarity = linear_kernel( tdfid_matrix, tdfid_matrix)


# Se crea la funcion de recomendación de 5 juegos recomendados para el usuario ingresado.
def recomendacion_usuario(id: int):
    if id not in Df_muestra['id'].values:
        return {'mensaje': 'No existe el id del usuario.'}
    titulo = Df_muestra.loc[Df_muestra['id'] == id, 'app_name'].iloc[0]
    sim_juegos = obtener_juegos_similares(titulo)
    
    return {'juegos recomendados': sim_juegos}

def obtener_juegos_similares(titulo: str, num_recomendaciones: int = 5):
    idx = Df_muestra[Df_muestra['app_name'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:num_recomendaciones + 1]]
    sim_juegos = Df_muestra['app_name'].iloc[sim_ind].values.tolist()
    
    return sim_juegos
recomendacion_usuario(220)

{'juegos recomendados': ['Half-Life 2',
  'Day of Defeat: Source',
  'Empire: Total War™',
  "Garry's Mod",
  'Day of Defeat: Source']}